In [7]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import re

In [8]:
whatcar_models = {
    "mg": [
        "https://www.whatcar.com/mg-motor-uk/mg3/hatchback/review/n17867",
        "https://www.whatcar.com/mg-motor-uk/zs/hatchback/review/n17318",
        "https://www.whatcar.com/mg-motor-uk/zs/hatchback/review/n20171",
        "https://www.whatcar.com/mg-motor-uk/hs/hatchback/review/n20298",
        "https://www.whatcar.com/mg-motor-uk/mg4/hatchback/review/n24916",
        "https://www.whatcar.com/mg-motor-uk/mgs5/estate/review/n27533",
        # "4-xpower",
        # "gs",
        # "5"
    ],
    "byd": [
        "https://www.whatcar.com/byd/dolphin/hatchback/review/n25891",
        "https://www.whatcar.com/byd/seal-u/estate/review/n26815",
        "https://www.whatcar.com/byd/atto-3/hatchback/review/n25563",
        "https://www.whatcar.com/byd/seal/saloon/review/n26074",
        "https://www.whatcar.com/byd/sealion-7/estate/review/n27254"
    ],
    "volkswagen": [
        "https://www.whatcar.com/volkswagen/touareg/4x4/review/n199",
        "https://www.whatcar.com/volkswagen/id3/hatchback/review/n21832",
        "https://www.whatcar.com/volkswagen/id4/estate/review/n22789",
        "https://www.whatcar.com/volkswagen/id5/coupe/review/n24518",
        "https://www.whatcar.com/volkswagen/id7/saloon/review/n26282",
        "https://www.whatcar.com/volkswagen/id7/estate/review/n27046",
        "https://www.whatcar.com/volkswagen/tiguan/estate/review/n17357",
        "https://www.whatcar.com/volkswagen/passat/estate/review/n29",
        "https://www.whatcar.com/volkswagen/golf/hatchback/review/n17371",
        "https://www.whatcar.com/volkswagen/t-roc/hatchback/review/n17376",
        "https://www.whatcar.com/volkswagen/t-cross/estate/review/n19039",
        "https://www.whatcar.com/volkswagen/polo/hatchback/review/n83"
    ],
    "renault": [
        "https://www.whatcar.com/renault/clio/hatchback/review/n97",
        "https://www.whatcar.com/renault/5/hatchback/review/n27187",
        "https://www.whatcar.com/renault/symbioz/estate/review/n27041",
        "https://www.whatcar.com/renault/megane-e-tech/hatchback/review/n24175",
        "https://www.whatcar.com/renault/austral/estate/review/n25066",
        "https://www.whatcar.com/renault/captur/hatchback/review/n20",
        "https://www.whatcar.com/renault/scenic-e-tech/estate/review/n26635",
        "https://www.whatcar.com/renault/rafale/estate/review/n26868",
        # "twingo",
        # "megane",
        # "kadjar",
        # "scenic",
        "https://www.whatcar.com/renault/arkana/estate/review/n23158",
        # "koleos",
        "https://www.whatcar.com/renault/zoe-van/van/review/n22211"
    ],
    "peugeot": [
        "https://www.whatcar.com/peugeot/208/hatchback/review/n30",
        "https://www.whatcar.com/peugeot/208/hatchback/review/n20228",
        "https://www.whatcar.com/peugeot/2008/estate/review/n110",
        "https://www.whatcar.com/peugeot/2008/estate/review/n20552",
        "https://www.whatcar.com/peugeot/308/hatchback/review/n17380",
        "https://www.whatcar.com/peugeot/308/hatchback/review/n26197",
        "https://www.whatcar.com/peugeot/308/estate/review/n17314",
        # "e-308-sw",
        "https://www.whatcar.com/peugeot/408/saloon/review/n25222",
        # "e-408",
        "https://www.whatcar.com/peugeot/3008/estate/review/n17363",
        "https://www.whatcar.com/peugeot/3008/estate/review/n26800",
        "https://www.whatcar.com/peugeot/5008/4x4/review/n17369",
        "https://www.whatcar.com/peugeot/5008/4x4/review/n26918",
        "https://www.whatcar.com/peugeot/rifter/estate/review/n23858",
        "https://www.whatcar.com/peugeot/traveller/mpv/review/n24152",
        "https://www.whatcar.com/peugeot/508/saloon/review/n17284"
    ],
    "fiat": [
        "https://www.whatcar.com/fiat/500/hatchback/review/n22352", 
        "https://www.whatcar.com/fiat/500/hatchback/review/n84", 
        # "500x", 
        # "600", 
        # "500l", 
        # "panda", 
        # "panda-cross", 
        # "panda-4x4", 
        "https://www.whatcar.com/fiat/600/hatchback/review/n26136", 
        # "tipo", 
        # "qubo", 
        "https://www.whatcar.com/fiat/500/sports/review/n22462"
    ],
    "bmw": [
        "https://www.whatcar.com/bmw/1-series/hatchback/review/n26", 
        "https://www.whatcar.com/bmw/2-series/hatchback/review/n78", 
        "https://www.whatcar.com/bmw/3-series/saloon/review/n91", 
        "https://www.whatcar.com/bmw/4-series/coupe/review/n62", 
        "https://www.whatcar.com/bmw/5-series/saloon/review/n63", 
        "https://www.whatcar.com/bmw/5-series/estate/review/n34",
        "https://www.whatcar.com/bmw/x1/estate/review/n25006", 
        "https://www.whatcar.com/bmw/ix1/estate/review/n25005", 
        "https://www.whatcar.com/bmw/x2/hatchback/review/n121", 
        "https://www.whatcar.com/bmw/ix2/hatchback/review/n26586", 
        "https://www.whatcar.com/bmw/x3/4x4/review/n42", 
        "https://www.whatcar.com/bmw/ix3/4x4/review/n22236", 
        "https://www.whatcar.com/bmw/x4/estate/review/n134", 
        "https://www.whatcar.com/bmw/x5/4x4/review/n38", 
        "https://www.whatcar.com/bmw/x6/4x4/review/n17306", 
        "https://www.whatcar.com/bmw/x7/4x4/review/n19102", 
        "https://www.whatcar.com/bmw/ix/4x4/review/n23569", 
        "https://www.whatcar.com/bmw/i4/hatchback/review/n23575", 
        "https://www.whatcar.com/bmw/i5/saloon/review/n26177", 
        "https://www.whatcar.com/bmw/i5/estate/review/n26831",  
    ],
}

In [9]:
# Set base headers
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36"
}

# Brand selezionati
BRANDS = ["mg", "byd", "volkswagen", "renault", "peugeot", "fiat", "bmw"]

BRAND_MODELS = {
    "mg": ["mg3", "zs", "zs-ev", "hs", "mg-4", "s5-ev", "4-xpower", "gs", "5"],
    "byd": ["dolphin", "seal-u", "atto-3", "seal", "sealion-7"],
    "volkswagen": ["touareg", "id3", "id4", "id5", "id7", "id7-tourer", "tiguan", "passat", "golf", "t-roc", "t-cross", "polo"],
    "renault": ["clio", "5-e-tech", "symbioz", "megane-etech-electric", "austral", "captur", "scenic-e-tech", "rafale", "twingo", "megane", "kadjar", "scenic", "arkana", "koleos", "zoe"],
    "peugeot": ["208", "e-208", "2008", "e-2008", "308", "e-308", "308-sw", "e-308-sw", "408", "e-408", "3008", "e-3008", "5008", "e-5008", "e-rifter", "e-traveller", "508"],
    "fiat": ["500", "500-electric", "500x", "600", "500l", "panda", "panda-cross", "panda-4x4", "600e", "tipo", "qubo", "500c"],
    "bmw": ["1-series", "2-series-active-tourer", "3-series", "4-series", "5-series", "5-series-touring", "x1", "ix1", "x2", "ix2", "x3", "ix3", "x4", "x5", "x6", "x7", "ix", "i4", "i5", "i5-touring"],
}


# Helper funzione per pulire testo
def clean_text(text: str):
    return re.sub(r'\s+', ' ', text.strip())


In [10]:
def scrape_carwow():
    results = []
    base_url = "https://www.carwow.co.uk/"
    for brand, model in BRAND_MODELS.items():
        for m in model:
            url = f"{base_url}{brand}/{m}"
            print(f"[Carwow] Scraping: {url}")
            try:
                res = requests.get(url, headers=HEADERS)
                soup = BeautifulSoup(res.content, "html.parser")
                
                # Titoli recensioni
                review_section = soup.find("section", class_="product-article-section")
                # review_section = soup.find("div", class_="content-block")
                if review_section:
                    summary = clean_text(review_section.text)
                else:
                    summary = "N/A"
                
                # rating_tag = soup.find("section", class_="cw-score__number")
                # rating = rating_tag.text.strip() if rating_tag else "N/A"

                results.append({
                    "brand": brand,
                    "model": m,
                    "text": summary,
                    "source": "Carwow"
                })
                time.sleep(1)
            except Exception as e:
                print(f"Error scraping {url}: {e}")
    return results

In [11]:
def scrape_whatcar():
    results = []
    for brand, model in whatcar_models.items():
        for m in model:
            print(f"[Whatcar] Scraping: {m}")
            model_name = m.split("/", maxsplit=5)[4]  # Estrai il nome del modello dalla URL
            try:
                res = requests.get(m, headers=HEADERS)
                soup = BeautifulSoup(res.content, "html.parser")
                
                # Estrai il div con le classi specificate
                review_section = soup.find("div", class_="Grid_col-xs-12__TJ7HS Grid_col-md-9__4qJt9")
                if review_section:
                    summary = clean_text(review_section.get_text())
                else:
                    summary = "N/A"

                results.append({
                    "brand": brand,
                    "model": model_name,
                    "text": summary,
                    "source": "Whatcar"
                })
                time.sleep(1)
            except Exception as e:
                print(f"Error scraping {m}: {e}")
    return results

In [12]:
def main():
    all_data = []
    all_data += scrape_carwow()
    all_data += scrape_whatcar()

    with open("recensioni_auto.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["brand", "model", "text", "source"])
        writer.writeheader()
        writer.writerows(all_data)

    print(f"\n✅ Dataset completato con {len(all_data)} recensioni.")

In [13]:
main()

[Carwow] Scraping: https://www.carwow.co.uk/mg/mg3
[Carwow] Scraping: https://www.carwow.co.uk/mg/zs
[Carwow] Scraping: https://www.carwow.co.uk/mg/zs-ev
[Carwow] Scraping: https://www.carwow.co.uk/mg/hs
[Carwow] Scraping: https://www.carwow.co.uk/mg/mg-4
[Carwow] Scraping: https://www.carwow.co.uk/mg/s5-ev
[Carwow] Scraping: https://www.carwow.co.uk/mg/4-xpower
[Carwow] Scraping: https://www.carwow.co.uk/mg/gs
[Carwow] Scraping: https://www.carwow.co.uk/mg/5
[Carwow] Scraping: https://www.carwow.co.uk/byd/dolphin
[Carwow] Scraping: https://www.carwow.co.uk/byd/seal-u
[Carwow] Scraping: https://www.carwow.co.uk/byd/atto-3
[Carwow] Scraping: https://www.carwow.co.uk/byd/seal
[Carwow] Scraping: https://www.carwow.co.uk/byd/sealion-7
[Carwow] Scraping: https://www.carwow.co.uk/volkswagen/touareg
[Carwow] Scraping: https://www.carwow.co.uk/volkswagen/id3
[Carwow] Scraping: https://www.carwow.co.uk/volkswagen/id4
[Carwow] Scraping: https://www.carwow.co.uk/volkswagen/id5
[Carwow] Scraping: 

In [1]:
import pandas as pd
import re
import ollama
import torch

In [2]:
# Carica il dataset e mostra le prime righe
df = pd.read_csv("recensioni_auto.csv")
print("\nAnteprima del dataset:")
df.head()


Anteprima del dataset:


,brand,model,text,source
0,mg,mg3,Is the MG3 a good car? The MG3 used to be a bi...,Carwow
1,mg,zs,Is the MG ZS a good car? The previous-generati...,Carwow
2,mg,zs-ev,Is the MG ZS EV a good car? The MG ZS EV is an...,Carwow
3,mg,hs,Is the MG HS a good car? The MG HS has at poin...,Carwow
4,mg,mg-4,Is the MG4 EV a good car? MG pulled an absolut...,Carwow


In [122]:
def extract_important_content(text):
    # Rimuovi frasi introduttive generiche
    text = re.sub(r'\bWhat Car\? says\.{0,3}', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\bIs the [^?]+\? ?', '', text, flags=re.IGNORECASE)

    # Rimuovi riferimenti a video e inviti alla lettura
    text = re.sub(r'\b[\w\s]*video review\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Read on to find out…', '', text, flags=re.IGNORECASE)

    # Rimuovi solo le frasi promozionali, senza mangiare testo prima/dopo
    text = re.sub(r'Sound interesting\?', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Check here for our best [^.]+\.', '', text, flags=re.IGNORECASE)
    text = re.sub(r'look for a used [^.]+\.', '', text, flags=re.IGNORECASE)
    text = re.sub(r'You could also check out other used [^.]+\.', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Want to sell your car online\? Carwow can help with that, too\.', '', text, flags=re.IGNORECASE)
    text = re.sub(r'Carwow can even help you sell your car online too\.', '', text, flags=re.IGNORECASE)
    text = re.sub(r'head on over to our .*? deals page to see how much money .*? can save you\.', '', text, flags=re.IGNORECASE)

    # Rimuovi firme
    text = re.sub(r'Mat Watson Expert Car Reviewer', '', text, flags=re.IGNORECASE)

    # Rimuovi blocchi con solo listini/offerte (ma non specifiche tecniche)
    text = re.sub(r'(New car deals|Leasing deals).*?(From £[0-9,]+)?', '', text, flags=re.IGNORECASE)

    # Rimuovi intestazioni vuote (es. "Overview", "Performance & drive", se isolate)
    text = re.sub(r'\b(Overview|Our Pick|Performance & drive|Strengths|Weaknesses|Verdict|Specification)\b\s*', '', text)

    # Pulisci spazi extra
    text = re.sub(r'\s+', ' ', text).strip()

    return text



In [123]:
for index, row in df.iterrows():
    original_text = row['text']
    cleaned_text = extract_important_content(original_text)
    df.at[index, 'text'] = cleaned_text
# Salva il dataset pulito
df.to_csv("recensioni_auto_pulite.csv", index=False, encoding="utf-8")
print("\nDataset pulito salvato come 'recensioni_auto_pulite.csv'.")


Dataset pulito salvato come 'recensioni_auto_pulite.csv'.


In [317]:
with open("system_information.txt", "r") as f:
    system_information = f.read()
system_information

'You are a sentiment analyzer tasked with classifying specific aspects of a car review.\n\n**Task**:\nGiven a full car review, extract sentiment for each of the following 14 categories:\n\n    - Drive\n    - Quality of interior\n    - Infotainment system\n    - Comfort\n    - Performance\n    - Handling\n    - Practicality\n    - Reliability\n    - Safety\n    - Quality of construction\n    - Noise\n    - Engine\n    - Price\n    - Overall\n\n**Instructions**:\n\n    - For each category, assign one of: Positive, Negative, Neutral, or Not mentioned (only if the category is truly not referenced).\n    - For Price, sentiment must be one of: very high, high, medium, or low.\n    - Use the Overall category to express the general sentiment toward the entire car.\n    - Include the Brand, Model, and Source of the review.\n    - Minimize the use of "Not mentioned" by inferring sentiment where context allows.\n    - Respond with a single line of output only.\n\n**MANDATORY FORMAT (read carefull

In [318]:
# Combine reviews from different sources for each (brand, model) pair

df = pd.read_csv("recensioni_auto.csv")

def combine_reviews(group):
    # If two reviews exist, append the second to the first (text and source)
    if len(group) == 2:
        combined_text = group.iloc[0]['text'] + " WhatCar: " + group.iloc[1]['text']
        combined_source = group.iloc[0]['source'] + " \\\\ " + group.iloc[1]['source']
        return pd.Series({
            'brand': group.iloc[0]['brand'],
            'model': group.iloc[0]['model'],
            'text': combined_text,
            'source': combined_source
        })
    else:
        # If only one review, keep as is
        return group.iloc[0][['brand', 'model', 'text', 'source']]

df_combined = (
    df.groupby(['brand', 'model'], as_index=False)
      .apply(combine_reviews)
      .reset_index(drop=True)
)

print(len(df_combined))
df_combined.head(50)

91


/tmp/ipykernel_45254/3111129283.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(combine_reviews)


,brand,model,text,source
0,bmw,1-series,Is the BMW 1 Series a good car? The BMW 1 Seri...,Carwow \\ Whatcar
1,bmw,2-series-active-tourer,Is the BMW 2 Series Active Tourer a good car? ...,Carwow \\ Whatcar
2,bmw,3-series,Is the BMW 3 Series a good car? The BMW 3 Seri...,Carwow \\ Whatcar
3,bmw,4-series,Is the BMW 4 Series a good car? The BMW 4 Seri...,Carwow \\ Whatcar
4,bmw,5-series,Is the BMW 5 Series a good car? Think of an al...,Carwow \\ Whatcar
5,bmw,5-series-tourer,What Car? says...What could possibly top the B...,Whatcar
6,bmw,5-series-touring,Is the BMW 5 Series Touring a good car? The BM...,Carwow
7,bmw,i4,Is the BMW i4 a good car? The BMW i4 is like a...,Carwow \\ Whatcar
8,bmw,i5,Is the BMW i5 a good car? The BMW i5 is a bit ...,Carwow \\ Whatcar
9,bmw,i5-touring,Is the BMW i5 Touring a good car? The BMW i5 T...,Carwow \\ Whatcar


In [319]:
reviews = []
for idx, row in df_combined.iterrows():
    brand = row['brand']
    model = row['model']
    text: str = row['text']
    source = row['source']
    review = f"{brand}, {model}, {text.strip()}, {source}"
    reviews.append(review)

reviews[:5]

["bmw, 1-series, Is the BMW 1 Series a good car? The BMW 1 Series is a posh family hatchback with a high-quality cabin and loads of cutting-edge tech. You may also be considering the Audi A3 or Mercedes A-Class if you’re shopping for a premium hatchback, and the 1 Series was updated in 2024 with a new look and an updated cabin. The 1 Series has always been the go-to if you want a premium hatchback with a sporty edge. It’s a bit like choosing a pair of Yeezys over some Dr. Marten loafers. There are very few small cars which feel as posh inside as the BMW 1 Series, and this was only improved during the 2024 update. All the materials feel soft and yielding and everything is really solid. Quality in here is much better than it is in a Mercedes A-Class. The tech you get inside is also really impressive. All cars get a huge curved display atop the dashboard comprising a 10.7-inch touchscreen and a 10.3-inch digital driver’s display, and it’s all very responsive and easy to use. Watch: BMW M1

In [320]:
# Creation of the dataframe
df_new = pd.DataFrame(
    columns = [
        'Brand', 
        'Model', 
        'Source', 
        'Drive', 
        'Quality of interior', 
        'Infotainment system', 
        'Comfort', 
        'Performance',
        'Handling', 
        'Practicality',  
        'Reliability', 
        'Safety', 
        'Quality of construction', 
        'Noise', 
        'Engine', 
        'Price',
        'Overall',
    ]
)

In [321]:
model_llm = ["deepseek-r1:8b", "llama3.1:8b-instruct-q5_K_M", "gemma3:4b", "gemma3:4b-it-q8_0"]

In [322]:
def extract_insights_ollama(text):
    print(f"\n[Ollama] Processing text{text}...")
    response = ollama.chat(
        model=model_llm[1],
        messages=[
            {'role': 'system', 'content': system_information},
            {'role': 'user', 'content': text}
        ],
    )
    # print(f"\n[Ollama] completed")
    
    return response['message']['content'].strip()

In [323]:
for i in range(len(reviews)):
    print(f"\n[Ollama] Elaborazione recensione {i + 1}/{len(reviews)}...")
    try:
        insights: str = extract_insights_ollama(reviews[i])
        print(f"[Ollama] Risposta grezza:\n{insights}")

        # Extract inside triple backticks
        match = re.search(r'```(.*?)```', insights, flags=re.DOTALL)
        if match:
            insights_clean = match.group(1).strip()
        else:
            print("[Ollama] Nessun blocco con backticks trovato, uso l'intero testo.")
            insights_clean = insights.strip()

        # Now split expecting 17 fields (16 pipes)
        result = insights_clean.split('||', 16)  # 16 splits = 17 parts
        print(f"[Ollama] Risultato pulito ({len(result)} campi):", result)

        # Retry if wrong number of fields
        if len(result) != 17:
            print("[Ollama] Formato non valido, secondo tentativo...")
            insights = extract_insights_ollama(reviews[i])
            match = re.search(r'```(.*?)```', insights, flags=re.DOTALL)
            if match:
                insights_clean = match.group(1).strip()
            else:
                insights_clean = insights.strip()
            result = insights_clean.split('||', 16)
            print(f"[Ollama] Secondo tentativo ({len(result)} campi):", result)

        if len(result) != 17:
            raise ValueError("Numero di campi non corretto (attesi 17).")

        # Store only the 17 fields
        df_new.loc[i] = [field.strip() for field in result]

        if i == 2:
            break

    except Exception as e:
        print(f"[Errore] Recensione {i + 1} fallita: {e}")


[Ollama] Elaborazione recensione 1/91...

[Ollama] Processing textbmw, 1-series, Is the BMW 1 Series a good car? The BMW 1 Series is a posh family hatchback with a high-quality cabin and loads of cutting-edge tech. You may also be considering the Audi A3 or Mercedes A-Class if you’re shopping for a premium hatchback, and the 1 Series was updated in 2024 with a new look and an updated cabin. The 1 Series has always been the go-to if you want a premium hatchback with a sporty edge. It’s a bit like choosing a pair of Yeezys over some Dr. Marten loafers. There are very few small cars which feel as posh inside as the BMW 1 Series, and this was only improved during the 2024 update. All the materials feel soft and yielding and everything is really solid. Quality in here is much better than it is in a Mercedes A-Class. The tech you get inside is also really impressive. All cars get a huge curved display atop the dashboard comprising a 10.7-inch touchscreen and a 10.3-inch digital driver’s dis

KeyboardInterrupt: 

In [ ]:
df_new.head()

,Brand,Model,Source,Drive,Quality of interior,Infotainment system,Comfort,Performance,Handling,Practicality,Reliability,Safety,Quality of construction,Noise,Engine,Price,Overall
0,BMW,1-Series,WhatCar?,Positive,Positive,Positive,Neutral,Positive,Positive,Medium,Positive,Positive,Positive,Not mentioned,Low,Very high,Very high
1,BMW,2-Series Active Tourer,Carwow - Whatcar,Positive,Neutral,Positive,Comfortable,Neutral,Easy to drive,Neutral,Practical,Not mentioned,Positive,Neutral,Low,Overall,Positive
2,bmw,3-series,Carwow - Whatcar,Positive,Neutral,Negative,Positive,Positive,Positive,Positive,Neutral,Positive,Not mentioned,Positive,Medium,Positive,Overall: Positive
